In [1]:
import requests
from obi_auth import get_token
from entitysdk import Client, ProjectContext, models
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    # LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    # Note: LOCAL_AUTHENTICATED is not supported by entitycore because the project/vlab ids do not exist in your keycloak groups

obi_one_api_url = "http://127.0.0.1:8100"

add_morphology_asset = False

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    project_id="2720f785-a3a2-4472-969d-19a53891c817"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "http://127.0.0.1:8000"
    add_morphology_asset = True
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730"
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6"
    # virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    # project_id="2720f785-a3a2-4472-969d-19a53891c817"
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Register circuit

In [2]:
# species = client.search_entity(
#     entity_type=models.Species, query={"name__ilike": "Homo sapiens"}
# ).one()

# subject = models.Subject(
#     name="my-subject", description="my-subject-description", sex="male", species=species
# )
# subject = client.register_entity(subject)

# brain_region = client.search_entity(entity_type=models.BrainRegion, query={"acronym": "CB"}).first()


# circuit = models.Circuit(
#     name="my-circuit",
#     description="my-circuit",
#     subject=subject,
#     brain_region=brain_region,
#     number_synapses=2,
#     number_neurons=5,
#     number_connections=10,
#     scale="microcircuit",
#     build_category="em_reconstruction",
# )

# circuit = client.register_entity(circuit)

# print(circuit)
# # 5b2368ce-f632-47a5-9d5d-3c1c4159a232

# Create SimulationForm using Cirucit

In [3]:
import obi_one as obi

# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(name="O1 Simulation", description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)))
sim_conf.add(sim_neuron_set, name='L1All')

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timestamps.ref, stim_duration=800, frequency=20, neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='VoltageRecording')

# Initialization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=obi.CircuitFromID(id_str="5b2368ce-f632-47a5-9d5d-3c1c4159a232"), 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

# Generate GridScan

In [4]:
import tempfile
import os

# Create a temporary directory
with tempfile.TemporaryDirectory() as temp_dir:
    print(f"Temporary directory created at: {temp_dir}")
    grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../obi-output/circuit_simulations/grid_scan')
    # grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    # grid_scan.coordinate_instances(display=True)
    grid_scan.execute(processing_method='generate', data_postprocessing_method='save', db_client=client)

Temporary directory created at: /var/folders/pp/r2gyck8j4n9dfb_x9vjslq740000gn/T/tmpdlqakeb3
[2025-06-19 17:23:32,104] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-19 17:23:32,104] INFO: No multiple value parameters found.
[2025-06-19 17:23:32,105] INFO: 
COORDINATE PARAMETERS
[2025-06-19 17:23:32,105] INFO: No coordinate parameters.
[2025-06-19 17:23:32,105] INFO: <entitysdk.client.Client object at 0x111e1a5a0>
[2025-06-19 17:23:32,105] INFO: 1. Initializing simulation campaign in the database...
[2025-06-19 17:23:32,105] INFO: -- Register SimulationCampaign Entity
[2025-06-19 17:23:32,106] INFO: -- Upload campaign_generation_config
[2025-06-19 17:23:32,106] INFO: -- Upload campaign_summary
initialize.circuit is a CircuitFromID instance.


AttributeError: 'Simulation' object has no attribute 'index'